In [10]:
import os
import sys

from os.path import join, pardir
sys.path.append(pardir)

from bids import BIDSLayout
from itertools import product, chain

from nipype import config
from nipype.pipeline.engine import Workflow, Node
from ica_wf import make_subject_ica_wf

import numpy as np
from nipype.interfaces.fsl.maths import TemporalFilter
from nipype.interfaces.fsl.model import MELODIC
from nipype.interfaces.fsl.preprocess import SUSAN
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.pipeline.engine import Workflow, Node

In [2]:
config.enable_debug_mode()

## Preproc & ICA Flow:

(For every dataset:)  
For every subject:  
For every run:  
For every task:  
For every space:  

      1. extract filenames of bold & mask file
      2. feed into preproc.py
      -> return ICs


### Calculate ICs

In [ ]:
import numpy as np
from nipype.interfaces.fsl.maths import TemporalFilter
from nipype.interfaces.fsl.model import MELODIC
from nipype.interfaces.fsl.preprocess import SUSAN
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.pipeline.engine import Workflow, Node

def calc_susan_thresh(boldfile, maskfile, timeax=0, median_factor=.75):
    """
    Calculate the median value within brainmask and multiply with fixed
    factor to get an estimate of the contrast between background and brain
    for FSL's SUSAN.
    """
    from nilearn.masking import apply_mask
    import numpy as np
    data = apply_mask(boldfile, maskfile)
    med = np.median(data.mean(axis=timeax))
    del data  # suspect memory leak
    return med * median_factor

def make_subject_ica_wf(): # boldfile, maskfile, outdir
    """
    Example Inputs:
        wf.inputs.inputspec.bold_file = '/../preproc_bold.nii.gz'
        wf.inputs.inputspec.mask_file = '/../brain_mask.nii.gz'
        wf.inputs.inputspec.tr = 1.5
        wf.inputs.inputspec.hpf = 120.0/tr
        wf.inputs.inputspec.fwhm = 4.0
        wf.inputs.inputspec.out_dir = '/results/melodic'
        wf.base_dir = '/..'
    
    Output:
        nipype workflow = combining precessing (smoothing/temporal filtering)
        and ICA for one functional run
    TODO: give TR as input (to this function) or infer from data?
    """
    #infosource = Node(IdentityInterface(fields=['subject_id']),
    #              name="infosource")
    #infosource.iterables = [('subject_id', subject_list)]
    
    #b.iterables = [("m", [1, 2]), ("n", [3, 4])]
    #b.synchronize = True
    
    # create input spec
    inputspec = Node(
        IdentityInterface(
            fields=['bold_file',
                    'mask_file',
                    'tr',
                    'hpf',
                    'fwhm',
                    'out_dir']
                    ),
            name="inputspec")
    #inputspec.iterables = [("bold_file", boldfile), ("mask_file", maskfile), ("out_dir", outdir)]
    #inputspec.synchronize = True
    
    # create node for smoothing
    calcthresh = Node(
        Function(
            function=calc_susan_thresh, input_names=['boldfile', 'maskfile'], 
            output_names=['smooth_thresh']),
            name='calcthresh'
    )
    susan = Node(SUSAN(), name='susan') # requires 
    # ... temporal filtering
    tfilt = Node(TemporalFilter(), name='tfilt')
    # ... ICA
    melodic = Node(MELODIC(out_all=True, no_bet=True, report=True), name='melodic')
    
    # connect nodes in workflow
    wf = Workflow(name='melodicwf')
    wf.connect([
        (inputspec, calcthresh, [('bold_file', 'boldfile'),
                                 ('mask_file', 'maskfile')]),
        (inputspec, susan, [('bold_file', 'in_file'),
                            ('fwhm', 'fwhm')]),
        (inputspec, tfilt, [('hpf', 'highpass_sigma')]),
        (inputspec, melodic, [('mask_file', 'mask'),
                              ('out_dir', 'out_dir'),
                              ('tr', 'tr_sec')]),
        (calcthresh, susan, [('smooth_thresh', 'brightness_threshold')]),
        (susan, tfilt, [('smoothed_file', 'in_file')]),
        (tfilt, melodic, [('out_file', 'in_files')])
    ])
    return wf

### Read datapaths

In [3]:
def get_datapaths(bids_layout, subject, session, run, task, space, outdir):
    """
    Extract mask and bold file paths for one subject for one task
    of one run for one type of space.
    
    Input: BIDSlayout, subject, ...  
    Output: bold_file, mask_file
    """
    # check if run and session are present
    run = None if run in ('0','00', '') else run
    session = None if session in ('0','00', '') else session
    # get paths
    bold_file = layout.get(
                        subject=subject,
                        run=run,
                        session=session,
                        task=task,
                        space=space,
                        extension='nii.gz',
                        suffix='bold',
                        return_type='filename'
                        )
    bold_file = [i for i in bold_file if "AROMA" not in i]
    
    mask_file = layout.get(
                        subject=subject,
                        run=run,
                        session=session,
                        task=task,
                        space=space,
                        extension='nii.gz',
                        suffix='mask',
                        return_type='filename'
                        )
    out_dir = join(outdir,
                   f'sub-{subject}') #,
                   #f'sub-{subject}_ses-{session}_task-{task}_run-{run}_space-{space}-melodic.nii.gz')
    return bold_file, mask_file, out_dir

In [4]:
def read_paths(bids_layout, subject="all", session="all", run="all", task="all", space="all",
               outdir="/LOCAL/jzerbe/temp_results/melodic"):
    """
    Check if all data paths or only specific paths are asked for and give them back.
    
    Input: BIDSlayout
    Optional Input: subject, session, run, task, space
    Output: one file with all bold and mask file paths as tuples
    """
    if all([param == "all" for param in (subject, session, run, task, space)]):
        subject = bids_layout.get(return_type='id', target='subject', desc='preproc')
        session = bids_layout.get(return_type='id', target='session', desc='preproc')
        run = bids_layout.get(return_type='id', target='session', desc='preproc')
        task = bids_layout.get(return_type='id', target='task', desc='preproc')
        space = bids_layout.get(return_type='id', target='space', desc='preproc')
    else:
        subject = subject
        session = session # TODO: for many runs/sessions, check if pybids gives just a number or a list
        run = run
        task = task
        space = space
    
    # check if run and session are present
    session = '0' if session == [] else session
    run = '0' if run == [] else run
    
    # all combinations
    combinations = list(product(subject, session, run, task, space))
    boldfiles_nested, maskfiles_nested, outdirs_nested = zip(*[
        get_datapaths(bids_layout, *params, outdir) for params in combinations
    ])
    boldfiles = [val for sublist in boldfiles_nested for val in sublist]
    maskfiles = [val for sublist in maskfiles_nested for val in sublist]
    outdirs = list(outdirs_nested) #= [val for sublist in outdirs_nested for val in sublist]
    
    return boldfiles, maskfiles, outdirs 

In [5]:
# create layout from BIDS dataset
bidsdata_dir = '/LOCAL/jzerbe/faces_vs_houses/ds002938'
layout = BIDSLayout(bidsdata_dir, derivatives=True)

In [6]:
# get all data paths
OUT_DIR = '/LOCAL/jzerbe/results/melodic' #'/LOCAL/jzerbe/temp_results/melodic' # later change to /bidsdata_dir + /melodic
#boldlist, masklist, outdirlist = read_paths(layout, outdir=OUT_DIR)

In [7]:
## testing: get data paths for one subject for certain parameters
subjects = ['13'] 
sessions = ['0'] # per subject
runs = ['0']
tasks = ['effort']
spaces = ['T1w']
boldlist, masklist, outdirlist = read_paths(layout, subjects, sessions, runs, tasks, spaces, outdir=OUT_DIR)
outdirlist

['/LOCAL/jzerbe/results/melodic/sub-13']

In [ ]:
boldlist

In [ ]:
# parameter for melodic workflow
TR = 1.5
HPF = 120./TR
FWHM = 4.0
BASE_DIR = '/LOCAL/jzerbe/temp_temp_results' # change to /bidsdata_dir?

In [8]:
def iterate_over_filepaths(bold_file, mask_file, out_dir):
    import os
    import sys
    
    from os.path import pardir
    sys.path.append(pardir)
    print(os.getcwd())
    print("##### ", pardir)
    from ica_wf import make_subject_ica_wf
    
    runwf = make_subject_ica_wf()
    
    runwf.inputs.inputspec.hpf = 80.
    runwf.inputs.inputspec.tr = 1.5
    runwf.inputs.inputspec.fwhm = 4.0
    runwf.base_dir = '/LOCAL/jzerbe/temporary_results'
    
    runwf.inputs.inputspec.bold_file = bold_file
    runwf.inputs.inputspec.mask_file = mask_file
    runwf.inputs.inputspec.out_dir = out_dir
    
    runwf.run()
    

In [11]:
#infosource = Node(IdentityInterface(fields=['subject_id']),
##              name="infosource")
##infosource.iterables = [('subject_id', subject_list)]
#    
##b.iterables = [("m", [1, 2]), ("n", [3, 4])]
##b.synchronize = True
#
#
#infosource = Node(IdentityInterface(fields=['bold_file', 'mask_file', 'out_dir']),
#                  name="infosource")
#infosource.inputs.bold_file = 
#infosource.inputs.mask_file = 
#infosource.inputs.out_dir = 
##infosource.iterables = [('subject_id', subject_list)]
#iden = Node(IdentityInterface(fields=['bold_file', 'mask_file', 'out_dir']), name="identity")
#iden.iterables = [("bold_file", boldlist), ('mask_file', masklist), ('out_dir', outdirlist)]

iteratefiles = Node(
        Function(
            function=iterate_over_filepaths, fields=['bold_file', 'mask_file', 'out_dir']),
            name='iteratefiles'
    )
iteratefiles.inputs.bold_file = boldlist
iteratefiles.inputs.mask_file = masklist
iteratefiles.inputs.out_dir = outdirlist
iteratefiles.iterables = [("bold_file", boldlist), ("mask_file", masklist), ("out_dir", outdirlist)]
iteratefiles.synchronize = True

iteratefiles.run()

#wf = Workflow(name="makeica")
#wf.base_dir = "/LOCAL/jzerbe/basedir"
#wf.connect([
#        (infosource, iteratefiles, [('bold_file', 'bold_file'),
#                                 ('mask_file', 'mask_file'),
#                                   ('out_dir', 'out_dir')])
#])
#
## Run it in parallel (one core for each smoothing kernel)
#wf.run('MultiProc', plugin_args={'n_procs': 3})
#

220510-17:16:47,735 nipype.workflow INFO:
	 [Node] Setting-up "iteratefiles" in "/tmp/tmpcp847rq1/iteratefiles".
220510-17:16:47,736 nipype.workflow DEBUG:
	 [Node] Not cached "/tmp/tmpcp847rq1/iteratefiles".
220510-17:16:47,738 nipype.utils DEBUG:
	 Removing contents of /tmp/tmpcp847rq1/iteratefiles
220510-17:16:47,739 nipype.workflow DEBUG:
	 [Node] Writing pre-exec report to "/tmp/tmpcp847rq1/iteratefiles/_report/report.rst"
220510-17:16:47,743 nipype.workflow INFO:
	 [Node] Executing "iteratefiles" <nipype.interfaces.utility.wrappers.Function>
/tmp/tmpcp847rq1/iteratefiles
#####  ..
220510-17:16:47,767 nipype.workflow INFO:
	 [Node] Finished "iteratefiles", elapsed time 0.001164s.
220510-17:16:47,768 nipype.workflow DEBUG:
	 Saving results file: '/tmp/tmpcp847rq1/iteratefiles/result_iteratefiles.pklz'
220510-17:16:47,768 nipype.workflow WARNING:
	 Storing result file without outputs
220510-17:16:47,769 nipype.workflow WARNING:
	 [Node] Error on "iteratefiles" (/tmp/tmpcp847rq1/iter

NodeExecutionError: Exception raised while executing Node iteratefiles.

Traceback (most recent call last):
  File "/LOCAL/jzerbe/testenv/lib/python3.6/site-packages/nipype/interfaces/utility/wrappers.py", line 78, in __init__
    self.inputs.function_str = getsource(function)
  File "/LOCAL/jzerbe/testenv/lib/python3.6/site-packages/nipype/utils/functions.py", line 12, in getsource
    return dedent(inspect.getsource(function))
  File "/usr/lib/python3.6/inspect.py", line 973, in getsource
    lines, lnum = getsourcelines(object)
  File "/usr/lib/python3.6/inspect.py", line 955, in getsourcelines
    lines, lnum = findsource(object)
  File "/usr/lib/python3.6/inspect.py", line 786, in findsource
    raise OSError('could not get source code')
OSError: could not get source code

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/LOCAL/jzerbe/testenv/lib/python3.6/site-packages/nipype/interfaces/base/core.py", line 398, in run

In [ ]:
# create a list with one workflow per bold/mask/output path

runwf = make_subject_ica_wf(boldlist, masklist, '/LOCAL/jzerbe/results')
    
runwf.inputs.inputspec.hpf = HPF
runwf.inputs.inputspec.tr = TR
runwf.inputs.inputspec.fwhm = FWHM
runwf.base_dir = BASE_DIR

#infosource.iterables = [('subject_id', subject_list)]
#b.iterables = [("m", [1, 2]), ("n", [3, 4])]
#b.synchronize = True
#runwf.inputs.inputspec.iterables = [('bold_file', boldlist), ('mask_file', masklist), ('out_dir', outdirlist)] 
#runwf.inputs.inputspec.iterables = [('bold_file', boldlist)]
#runwf.inputs.inputspec.bold_file = boldlist
#runwf.inputs.inputspec.mask_file = masklist
#runwf.inputs.inputspec.out_dir = outdirlist
#runwf.inputs.inputspec.synchronize = True
#runwfs.append(runwf)

In [ ]:
#dataset_wf = Workflow(name='dataset_wf')
#dataset_wf.add_nodes(runwfs)


# Write graph of type orig
#runwf.write_graph(graph2use='flat', dotfilename='./graph_orig.dot')

# Visualize graph
#from IPython.display import Image
#Image(filename="graph_orig.png")

runwf.run()